In [1]:
import time
import numpy as np
import DRL.takeoff_dqn as tfdqn
import LLCsimple
import LLC 
import scaffold.fgdata as dfer
import scaffold.pidpilot as PID
import datetime
import fgmodule.fgenv as fgenv
import pandas as pd
import os
from scaffold.timer import gettime

'''
##########controlframe############


##########state_dict##############
{'aileron': 0.0, 'elevator': 0.0, 'rudder': -0.026, 'flaps': 0.0, 'throttle0': 0.6, 'throttle1': 0.6, 'vsi-fpm': 0.0, 'alt-ft': -372.808502, 'ai-pitch': 0.401045, 'ai-roll': 0.050598, 'ai-offset': 0.0, 'hi-heading': 80.568947, 'roll-deg': 0.050616, 'pitch-deg': 0.401055, 'heading-deg': 90.013458, 'airspeed-kt': 71.631187, 'speed-north-fps': -0.021637, 'speed-east-fps': 119.609383, 'speed-down-fps': -0.071344, 'uBody-fps': 119.606964, 'vBody-fps': -0.005778, 'wBody-fps': 0.765776, 'north-accel-fps_sec': 0.118887, 'east-accel-fps_sec': 5.870498, 'down-accel-fps_sec': -0.003219, 'x-accel-fps_sec': 6.095403, 'y-accel-fps_sec': -0.148636, 'z-accel-fps_sec': -32.113453, 'latitude': 21.325245, 'longitude': -157.93947, 'altitude': 22.297876, 'crashed': 0.0}
'''

LLC_FEATURE_BOUNDS = {
    'aileron': [-1, 1],  # 副翼 控制飞机翻滚 [-1,1] left/right
    'rudder': [-1, 1],  # 方向舵 控制飞机转弯（地面飞机方向控制） 0 /enter
    'pitch-deg': [-90., 90.],  # 飞机俯仰角
    'roll-deg': [-180., 180.],  # 飞机滚转角
    'heading-deg': [0., 360.],  # 飞机朝向
}
LLC_GOAL_BOUNDS = {
    'pitch-deg': [-90., 90.],  # 飞机俯仰角
    'roll-deg': [-180., 180.],  # 飞机滚转角
    'heading-deg': [0., 360.],  # 飞机朝向
}
goals = {
    'pitch-deg': 0.,  # 飞机俯仰角
    'roll-deg': 0.,  # 飞机滚转角
    'heading-deg': 90.,  # 飞机朝向
}

LLC_ACTION_BOUNDS = {
    'aileron': [-1, 1],  # 副翼 控制飞机翻滚 [-1,1] left/right
    'rudder': [-1, 1],  # 方向舵 控制飞机转弯（地面飞机方向控制） 0 /enter
}

def fgstart(fg2client_addr=("127.0.0.1", 5700), client2fg_addr=("127.0.0.1", 5701), telnet_addr=("127.0.0.1", 5555)):
    '''
    简化FG通信启动过程
    ---
    Inputs:
        - fg2client_addr
        - client2fg_addr
        - telnet_addr
    Outputs:
        - myfgenv:初始话完成的fgenv类

    '''
    myfgenv = fgenv.fgenv(telnet_addr, fg2client_addr, client2fg_addr)
    # initial_state = myfgenv.initial()
    myfgenv.initial()

    return myfgenv


MAX_EPISODE = 1000
MAX_EP_STEPS = 200

GAMMA = 0.9
LR_A = 0.001    # learning rate for actor
LR_C = 0.01     # learning rate for critic

##
epoch = 1000
step = 220

In [2]:
myfgenv = fgstart()

# bounds = {
#     'rudder': [-1, 1],  # 方向舵 控制飞机转弯（地面飞机方向控制） 0 /enter
# }

flylog saving path check pass!
send data to 127.0.0.1:5701 !
fgudp is ready!!!
Bind UDP on 127.0.0.1:5700 !
save log to data/flylog
auto start完成 16 : 16 : 38
fgudp stop begin! waiting for 2 thread quit
send data to 127.0.0.1:5701 stop!UDP rece on 127.0.0.1:5700 stop!

reset完成 16 : 16 : 39
flylog saving path check pass!
send data to 127.0.0.1:5701 !
fgudp is ready!!!
Bind UDP on 127.0.0.1:5700 !
save log to data/flylog
auto start完成 16 : 17 : 34


In [2]:
myllc = LLCsimple.LLC(LLC_FEATURE_BOUNDS,LLC_GOAL_BOUNDS,LLC_ACTION_BOUNDS)

In [5]:
state = myfgenv.replay("sky")

replay to sky 完成!time:16 : 18 : 3
auto start完成 16 : 18 : 6


In [4]:
state = {'aileron': 0.0,
 'elevator': 0.0,
 'rudder': 0.0,
 'flaps': 0.0,
 'throttle0': 0.0,
 'throttle1': 0.0,
 'vsi-fpm': 0.0,
 'alt-ft': -370.319122,
 'ai-pitch': 0.534232,
 'ai-roll': 0.277171,
 'ai-offset': 0.0,
 'hi-heading': 83.792358,
 'roll-deg': 0.056131,
 'pitch-deg': 0.467942,
 'heading-deg': 90.00074,
 'airspeed-kt': 1.532065,
 'speed-north-fps': -0.000281,
 'speed-east-fps': -0.007067,
 'speed-down-fps': 0.000105,
 'uBody-fps': -0.007067,
 'vBody-fps': 0.000281,
 'wBody-fps': 4.7e-05,
 'north-accel-fps_sec': -1.4e-05,
 'east-accel-fps_sec': -0.009769,
 'down-accel-fps_sec': -0.000142,
 'x-accel-fps_sec': 0.252822,
 'y-accel-fps_sec': -0.031483,
 'z-accel-fps_sec': -32.151367,
 'latitude': 21.325247,
 'longitude': -157.943137,
 'altitude': 21.358186,
 'crashed': 0.0}

In [4]:
goal = goals

In [5]:
myllc.goals

{'pitch-deg': [-90.0, 90.0],
 'roll-deg': [-180.0, 180.0],
 'heading-deg': [0.0, 360.0]}

In [6]:
state

{'aileron': 0.0,
 'elevator': 0.0,
 'rudder': 0.0,
 'flaps': 0.0,
 'throttle0': 0.21,
 'throttle1': 0.21,
 'vsi-fpm': 0.0,
 'alt-ft': -373.07193,
 'ai-pitch': 0.444673,
 'ai-roll': 0.054528,
 'ai-offset': 0.0,
 'hi-heading': 80.698265,
 'roll-deg': 0.054527,
 'pitch-deg': 0.444676,
 'heading-deg': 90.142323,
 'airspeed-kt': 17.874088,
 'speed-north-fps': -0.068237,
 'speed-east-fps': 27.643753,
 'speed-down-fps': -0.025079,
 'uBody-fps': 27.643202,
 'vBody-fps': -0.000252,
 'wBody-fps': 0.189458,
 'north-accel-fps_sec': -0.014251,
 'east-accel-fps_sec': 1.868108,
 'down-accel-fps_sec': -0.001648,
 'x-accel-fps_sec': 2.117626,
 'y-accel-fps_sec': -0.020975,
 'z-accel-fps_sec': -32.138405,
 'latitude': 21.325246,
 'longitude': -157.94253,
 'altitude': 21.546366,
 'crashed': 0.0}

In [9]:
action ,_ = myllc.choose_action(state,goal)
print(LLCsimple.llc_reward(state , goal, old_action, action, 0))
print(action)
print(old_action)
old_action = action

-0.49682351424715865
[-0.5381245  -0.25378835]
[-0.64095265  0.4908171 ]


In [9]:
import importlib
importlib.reload(LLCsimple)

<module 'LLCsimple' from 'C:\\Users\\niebu\\Desktop\\FlightGear\\FGAutopilot\\FGAutopilot\\LLCsimple.py'>

In [6]:
action = np.array([0.,0.])
old_action = np.array([0.,0.])

In [14]:
 for e in range(epoch):
        state = myfgenv.replay("sky")
        time.sleep(2)

        next_goal = goal

        ep_reward = 0

        for s in range(step):
        
            
            old_action = action 
            action,action_true = myllc.choose_action(state,goal)
            
            elevator = -0.01*(0.0-state['pitch-deg'])

            action_frame = dfer.action2frame((action_true[0],elevator,action_true[1],0.6,0.6))
            
            next_state, reward , done , info = myfgenv.step(action_frame) 
            
            r_ = LLCsimple.llc_reward(state , goal,old_action,action ,reward)
            
            next_goal = goals
            myllc.learn(state, goal, r_, action,next_state , next_goal)
            
            state = next_state
            goal = next_goal

            ep_reward += r_
            if done:
                print('Episode:', e, ' Reward: %i' %
                      int(ep_reward), 'Explore: %.2f' % myllc.var, )
                break
            
            if s == step-1:
                print('Episode:', e, ' Reward: %i' %
                      int(ep_reward), 'Explore: %.2f' % myllc.var, )
                # if ep_reward > -300:RENDER = True
                break
            time.sleep(0.1)

replay to sky 完成!time:21 : 27 : 16
auto start完成 21 : 27 : 19
Episode: 0  Reward: -238 Explore: 0.04
replay to sky 完成!time:21 : 27 : 46
auto start完成 21 : 27 : 49
Episode: 1  Reward: -234 Explore: 0.04
replay to sky 完成!time:21 : 28 : 16
auto start完成 21 : 28 : 19
Episode: 2  Reward: -243 Explore: 0.04
replay to sky 完成!time:21 : 28 : 46
auto start完成 21 : 28 : 49
Episode: 3  Reward: -238 Explore: 0.04
replay to sky 完成!time:21 : 29 : 16
auto start完成 21 : 29 : 19
Episode: 4  Reward: -245 Explore: 0.04
replay to sky 完成!time:21 : 29 : 45
auto start完成 21 : 29 : 48
Episode: 5  Reward: -240 Explore: 0.04
replay to sky 完成!time:21 : 30 : 15
auto start完成 21 : 30 : 18
Episode: 6  Reward: -239 Explore: 0.04
replay to sky 完成!time:21 : 30 : 45
auto start完成 21 : 30 : 48
Episode: 7  Reward: -250 Explore: 0.04
replay to sky 完成!time:21 : 31 : 15
auto start完成 21 : 31 : 18
Episode: 8  Reward: -241 Explore: 0.04
replay to sky 完成!time:21 : 31 : 45
auto start完成 21 : 31 : 48
Episode: 9  Reward: -238 Explore: 0.04


Episode: 81  Reward: -256 Explore: 0.02
replay to sky 完成!time:22 : 8 : 17
auto start完成 22 : 8 : 20
Episode: 82  Reward: -232 Explore: 0.02
replay to sky 完成!time:22 : 8 : 47
auto start完成 22 : 8 : 50
Episode: 83  Reward: -228 Explore: 0.02
replay to sky 完成!time:22 : 9 : 17
auto start完成 22 : 9 : 20
Episode: 84  Reward: -240 Explore: 0.02
replay to sky 完成!time:22 : 9 : 47
auto start完成 22 : 9 : 50
Episode: 85  Reward: -240 Explore: 0.02
replay to sky 完成!time:22 : 10 : 17
auto start完成 22 : 10 : 20
Episode: 86  Reward: -229 Explore: 0.02
replay to sky 完成!time:22 : 10 : 47
auto start完成 22 : 10 : 50
Episode: 87  Reward: -262 Explore: 0.01
replay to sky 完成!time:22 : 11 : 17
auto start完成 22 : 11 : 20
Episode: 88  Reward: -239 Explore: 0.01
replay to sky 完成!time:22 : 11 : 48
auto start完成 22 : 11 : 51
Episode: 89  Reward: -210 Explore: 0.01
replay to sky 完成!time:22 : 12 : 18
auto start完成 22 : 12 : 21
Episode: 90  Reward: -237 Explore: 0.01
replay to sky 完成!time:22 : 12 : 48
auto start完成 22 : 12 : 5

auto start完成 22 : 48 : 23
Episode: 162  Reward: -255 Explore: 0.01
replay to sky 完成!time:22 : 48 : 50
auto start完成 22 : 48 : 53
Episode: 163  Reward: -240 Explore: 0.01
replay to sky 完成!time:22 : 49 : 20
auto start完成 22 : 49 : 23
Episode: 164  Reward: -217 Explore: 0.01
replay to sky 完成!time:22 : 49 : 50
auto start完成 22 : 49 : 53
Episode: 165  Reward: -243 Explore: 0.01
replay to sky 完成!time:22 : 50 : 20
auto start完成 22 : 50 : 23
Episode: 166  Reward: -230 Explore: 0.01
replay to sky 完成!time:22 : 50 : 50
auto start完成 22 : 50 : 53
Episode: 167  Reward: -244 Explore: 0.01
replay to sky 完成!time:22 : 51 : 20
auto start完成 22 : 51 : 23
Episode: 168  Reward: -237 Explore: 0.01
replay to sky 完成!time:22 : 51 : 50
auto start完成 22 : 51 : 53
Episode: 169  Reward: -239 Explore: 0.01
replay to sky 完成!time:22 : 52 : 20
auto start完成 22 : 52 : 23
Episode: 170  Reward: -210 Explore: 0.01
replay to sky 完成!time:22 : 52 : 50
auto start完成 22 : 52 : 53
Episode: 171  Reward: -261 Explore: 0.01
replay to sky 完

KeyboardInterrupt: 

In [15]:
myllc.save("modelckpt/LLCsimple_DDPG/LLCsimple_DDPG.ckpt")

In [38]:
myllc.ddpg.pointer

25081

In [37]:
t1= time.clock()
myllc.learn(state, goal, r_, action,next_state , next_goal)
print(time.clock()-t1)

0.0027973283603017762
